<a href="https://colab.research.google.com/github/Preciuse/Coursera_Capstone/blob/main/Capstone_Project_Week_1_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)

## Introduction: Business Problem <a name="introduction"></a>

The recent Covid outbreak across the world has impacted all aspects of our daily lives. However, an enterprising entrepreneur has decided to capitalise on the uncertainty and open an office for their new startup in a US state. Unfortunately, a large number of their employees are 'at-risk': thus, the entrepreneur seeks to find the safest state in terms of covid cases.

## Data <a name="data"></a>


The data we'll be using is sourced from the NY Times, linked below. The historical data used in this report is from 1 Jan 2020 to 2 Jun 2021, though we'll only be looking at the last 2 months (i.e. 1 Apr 2021 - 2 Jun 2021) for this analysis.

Source: https://github.com/nytimes/covid-19-data

Historical Data for US -https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv

Historical Data for States - https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv

Historical Data by County - https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Load historical covid data for the US, as well as data per state and per county
us_data = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv").dropna()
us_state_data = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv").dropna()
us_county_data = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv").dropna()

In [ ]:
# As we're working with only the 50 states and D.C., we drop everything else.
us_state_data = us_state_data.query('state != "Northern Mariana Islands" & state != "Guam" & state != "Puerto Rico" & state != "Virgin Islands"')
us_county_data = us_county_data.query('state != "Northern Mariana Islands" & state != "Guam" & state != "Puerto Rico" & state != "Virgin Islands"')

In [ ]:
# Note that the data is given in cumulative form, so there were no new cases or deaths for in, say, Wyoming after the 28th of May
us_state_data[us_state_data.state == "Wyoming"].tail(10)

,date,state,fips,cases,deaths
24653,2021-05-24,Wyoming,56,59797,713
24708,2021-05-25,Wyoming,56,59870,719
24763,2021-05-26,Wyoming,56,59961,719
24818,2021-05-27,Wyoming,56,60048,719
24873,2021-05-28,Wyoming,56,60144,719
24928,2021-05-29,Wyoming,56,60144,719
24983,2021-05-30,Wyoming,56,60144,719
25038,2021-05-31,Wyoming,56,60144,719
25093,2021-06-01,Wyoming,56,60364,720
25148,2021-06-02,Wyoming,56,60433,720


In [ ]:
# Limit the data to the start of this year (1 Jan 2021 ) to end of the data set, at 1 Jun 2021
us_data = us_data[us_data.date >= '2021-01-01']
us_state_data = us_state_data[us_state_data.date >= '2021-01-01']
us_county_data = us_county_data[us_county_data.date >= '2021-01-01']
us_state_data.head()

,date,state,fips,cases,deaths
16734,2021-01-01,Alabama,1,365747,4872
16735,2021-01-01,Alaska,2,46740,198
16736,2021-01-01,Arizona,4,530267,9015
16737,2021-01-01,Arkansas,5,229442,3711
16738,2021-01-01,California,6,2345726,26236


Plotly's US maps require state codes rather than state names, so we'll need to replace these in the dataframe.

In [ ]:
# Dictionary to replace state names with their code, in order to map the states: sourced from https://gist.github.com/rogerallen/1583593 with thanks
us_state_abbrev = {
'Alabama': 'AL','Alaska': 'AK','American Samoa': 'AS','Arizona': 'AZ','Arkansas': 'AR','California': 'CA','Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE','District of Columbia': 'DC','Florida': 'FL','Georgia': 'GA','Guam': 'GU','Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS','Missouri': 'MO','Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM','New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Northern Mariana Islands':'MP','Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR','Pennsylvania': 'PA','Puerto Rico': 'PR','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Vermont': 'VT','Virgin Islands': 'VI','Virginia': 'VA','Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY'
}
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

In [ ]:
us_state_data_coded = us_state_data.copy(deep=True)
us_state_data_coded['state'].replace(us_state_abbrev, inplace=True)
us_state_data_coded.dropna().head()

,date,state,fips,cases,deaths
16734,2021-01-01,AL,1,365747,4872
16735,2021-01-01,AK,2,46740,198
16736,2021-01-01,AZ,4,530267,9015
16737,2021-01-01,AR,5,229442,3711
16738,2021-01-01,CA,6,2345726,26236


In [ ]:
us_state_data_jun = us_state_data_coded.copy(deep=True)
us_state_data_jun = us_state_data_jun[us_state_data_jun.date == '2021-06-02']
us_state_data_jun.head()

,date,state,fips,cases,deaths
25094,2021-06-02,AL,1,544598,11167
25095,2021-06-02,AK,2,69773,352
25096,2021-06-02,AZ,4,882369,17648
25097,2021-06-02,AR,5,341692,5835
25098,2021-06-02,CA,6,3791824,63287


Let's take a look at the total number of cases that each state and territory has accumulated.

In [ ]:
import plotly.graph_objects as go

# Mapping the # of cumulative cases as of the 2nd of June, 2021, by state
fig = go.Figure(data=go.Choropleth(
    locations=us_state_data_jun['state'], # Spatial coordinates
    z = us_state_data_jun['cases'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Blues',
    colorbar_title = "# of cases",
))

fig.update_layout(
    title_text = 'Cumulative Covid Cases on 2021-06-02 by State',
    geo_scope='usa', # limit map scope to USA
)

fig.show()

However, the total number of cases doesn't tell us the whole picture - a state's population will affect how many cases they have, as well as the steps they've taken since the pandemic started. Let's look at the number of new cases that have popped up since the start of April, so about 2 months ago. First, we'll have to operate on the data.

In [ ]:
# New dataframe for data on the 1st of April 2021
us_state_data_apr = us_state_data_coded.copy(deep=True)
us_state_data_apr = us_state_data_apr[us_state_data_apr.date == '2021-04-01']

us_state_data_apr.head()

,date,state,fips,cases,deaths
21684,2021-04-01,AL,1,515866,10553
21685,2021-04-01,AK,2,62785,299
21686,2021-04-01,AZ,4,842200,16977
21687,2021-04-01,AR,5,330609,5636
21688,2021-04-01,CA,6,3671562,59395


In [ ]:
# For each state, find the difference in # of cases from April to June, and save it as a dataframe
states_list = us_state_data_jun["state"].unique()
new_cases_apr_jun_list = []

for state in states_list:
  new_cases_apr_jun_list.append(int(us_state_data_jun[us_state_data_jun.state == state].cases) - int(us_state_data_apr[us_state_data_apr.state == state].cases))

df_new_cases_jun_apr = pd.DataFrame({"state": states_list,
                                     "new cases": new_cases_apr_jun_list})

df_new_cases_jun_apr.head()

,state,new cases
0,AL,28732
1,AK,6988
2,AZ,40169
3,AR,11083
4,CA,120262


In [ ]:
# Mapping the # of cases from April to June, by state
fig = go.Figure(data=go.Choropleth(
    locations=df_new_cases_jun_apr['state'], # Spatial coordinates
    z = df_new_cases_jun_apr['new cases'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Blues',
    colorbar_title = "# of new cases, APR to JUN",
))

fig.update_layout(
    title_text = 'Number of Covid Cases from APR to JUN by State',
    geo_scope='usa', # limit map scope to USA
)

fig.show()

As we can see, Florida and some north-eastern states and territories have the most new cases in the last 2 months. Again, this doesn't tell the whole story - we'll need to factor in each state's population to get a better picture as to the extent of covid cases for each state.

The most recent US population data seems to be from 2019, so we'll have to extrapolate it to 2021.

In [ ]:
us_pop_data = pd.read_excel("https://www2.census.gov/programs-surveys/popest/tables/2010-2019/state/totals/nst-est2019-01.xlsx", header=3)
us_pop_data = us_pop_data.dropna()
# As the data contains the population estimates for the entire US and four quadrants, we'll need to remove the first five columns.
us_pop_data = us_pop_data.iloc[5:]

In [ ]:
# Refresh the df index after removing five rows, and rename the first column
us_pop_data = us_pop_data.reset_index(drop=True)
us_pop_data = us_pop_data.rename(columns={"Unnamed: 0": "state"})
# Remove Puerto Rico
us_pop_data = us_pop_data.query('state != "Puerto Rico"')
us_pop_data.head()

,state,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,.Alabama,4779736.0,4780125.0,4785437.0,4799069.0,4815588.0,4830081.0,4841799.0,4852347.0,4863525.0,4874486.0,4887681.0,4903185.0
1,.Alaska,710231.0,710249.0,713910.0,722128.0,730443.0,737068.0,736283.0,737498.0,741456.0,739700.0,735139.0,731545.0
2,.Arizona,6392017.0,6392288.0,6407172.0,6472643.0,6554978.0,6632764.0,6730413.0,6829676.0,6941072.0,7044008.0,7158024.0,7278717.0
3,.Arkansas,2915918.0,2916031.0,2921964.0,2940667.0,2952164.0,2959400.0,2967392.0,2978048.0,2989918.0,3001345.0,3009733.0,3017804.0
4,.California,37253956.0,37254519.0,37319502.0,37638369.0,37948800.0,38260787.0,38596972.0,38918045.0,39167117.0,39358497.0,39461588.0,39512223.0


However, the data stops at 2019, so we'll need to extrapolate to midway through 2021. We can find the average yearly rate of growth from 1 Apr 2010 to 1 Jul 2019, and forecast for population at 1 Jun 2021.

In [ ]:
# Verify that our data has the right number of states (50 + D.C.)
print(df_new_cases_jun_apr.shape)
print(us_pop_data.shape)

(51, 2)
(51, 13)


In [ ]:
import math

projected_pop_2021 = []

for i in us_pop_data.index:
  # 1 Apr 2010 to 1 Jul 2019 is 9 and 1/3 years
  growth_10_19 = int(us_pop_data.iloc[i, -1]) / int(us_pop_data.iloc[i, 3])
  # 1 over 9 + 1/3 = 3/28
  growth_10_19_yearly = math.pow(growth_10_19, 3/28)
  # 1 Jul 2019 to 1 Jun 2021 is 1 and 11/12 years
  extrap_pop_2021 = us_pop_data.iloc[i, -1] * pow(growth_10_19_yearly, 23/24)
  rounded_projection = int(extrap_pop_2021)
  projected_pop_2021.append(rounded_projection)

print(projected_pop_2021)

[4915438, 733380, 7374660, 3027820, 39744537, 5837231, 3563870, 981717, 716972, 21768004, 10715054, 1421312, 1810897, 12654626, 6757549, 3165981, 2919033, 4480126, 4659633, 1345926, 6072709, 6926900, 9998152, 5674524, 2976724, 6152139, 1077135, 1945510, 3121814, 1364199, 8890729, 2100171, 19459081, 10586709, 771647, 11704587, 3977776, 4258852, 12811351, 1059917, 5204508, 892009, 6879786, 29411616, 3253792, 623795, 8589885, 7710587, 1785890, 5836155, 580244]


Now, let's find the number of new cases from Apr to Jun as a percentage of each state's population, and save it to a dataframe.

In [ ]:
pcent_new_cases_list = []

for i in range(0,51):
  pcent_new_cases = new_cases_apr_jun_list[i] / projected_pop_2021[i]
  pcent_new_cases_list.append(pcent_new_cases*100) # multiply by 100 to convert from decimal to percentage

percentage_new_cases_apr_jun_df = pd.DataFrame({"state": states_list,
                                                "% new cases": pcent_new_cases_list})

In [ ]:
# Mapping the # of cases from April to June, by state
fig = go.Figure(data=go.Choropleth(
    locations=percentage_new_cases_apr_jun_df['state'], # Spatial coordinates
    z = percentage_new_cases_apr_jun_df['% new cases'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Blues',
    colorbar_title = "% of new cases of state population",
))

fig.update_layout(
    title_text = "Percentage of new covid cases, from APR to JUN, of a state's population",
    geo_scope='usa', # limit map scope to USA
)

fig.show()

As we can see, the state with the highest percentage of its inhabitants contracting covid from April to June is Michigan. Compared to the previous maps, California has a very low percentage, even though it had a high number of absolute cases: its even larger population drags the % down. Other states with high percentages include Minnestoa, Delaware, Colorado, and Pennsylvania.

In [ ]:
percentage_new_cases_apr_jun_df.sort_values(by=["% new cases"], ascending=False).head()

,state,% new cases
22,MI,2.397393
23,MN,1.409687
7,DE,1.397857
5,CO,1.383533
38,PA,1.357156
